In [1]:
import pandas as pd
import os
import subprocess
import glob
import uproot
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import awkward as ak
import mplhep as hep

In [2]:
def add_all_line(df, line):
    all_strings = ''
    for strings in df.iloc[line]:
        if type(strings) is str:
            all_strings += strings + '     '
    return all_strings


# resolved

In [7]:
cat_name = ['cat34FHlowpurity', 'cat34FHhighpurity','cat34SLhighpurity','cat34SLlowpurity']
mass_name=['MX250_MH125', 'MX260_MH125','MX270_MH125','MX280_MH125', 'MX300_MH125', 'MX320_MH125', 'MX350_MH125', 'MX400_MH125', 'MX450_MH125', 'MX500_MH125', 'MX550_MH125', 'MX600_MH125', 'MX650_MH125', 'MX700_MH125', 'MX750_MH125', 'MX800_MH125', 'MX850_MH125','MX1000_MH125']
# mass_name=['MX500_MH125']
#combine command
getlimit=[]
Combinecommand=[]
txttoroot=[]
cd = []
for cat in cat_name:
    for mass in mass_name: 
        print(mass,cat)
        mass_dir = "datacard_" + mass
        FHSLcat_path="/afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/2018datacard/"+mass_dir+"/Datacard_"+mass+"_2018_combineFHSL_"+cat+".txt"
        bbggcat_path="/afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/2018datacard/"+mass_dir+"/Datacard_"+mass+"_2018_bbgg_"+cat+".txt"
        zzggcat_path="/afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/2018datacard/"+mass_dir+"/Datacard_"+mass+"_2018_zzgg_"+cat+".txt"
        FHSLws_path="/eos/cms/store/group/phys_b2g/shsong/flashggws/cat34/2018/"+mass+"/ws_gghh_"+mass+"_2018_combineFHSL_"+cat
        bbggws_path="/eos/cms/store/group/phys_b2g/shsong/flashggws/cat34/2018/"+mass+"/ws_gghh_"+mass+"_2018_bbgg_"+cat
        zzggws_path="/eos/cms/store/group/phys_b2g/shsong/flashggws/cat34/2018/"+mass+"/ws_gghh_"+mass+"_2018_zzgg_"+cat
        FHSLsignalmodel= glob.glob(os.path.join(FHSLws_path, 'CMS-HGG_sigfit_packaged_*.root')) 
        bbggsignalmodel= glob.glob(os.path.join(bbggws_path, 'CMS-HGG_sigfit_packaged_*.root'))
        zzggsignalmodel= glob.glob(os.path.join(zzggws_path, 'CMS-HGG_sigfit_packaged_*.root'))
        FHSLsignaltxt = os.path.getsize(FHSLcat_path) > 1000
        bbggsignaltxt = os.path.getsize(bbggcat_path) > 1000
        zzggsignaltxt = os.path.getsize(zzggcat_path) > 1000
        if len(FHSLsignalmodel)>0:
            if FHSLsignaltxt:
                FHSLsignalmodel = True
            else:
                FHSLsignalmodel = False
        else:
            FHSLsignalmodel = False
        if len(bbggsignalmodel)>0:
            if bbggsignaltxt:
                bbggsignalmodel = True
            else:
                bbggsignalmodel = False
        else:
            bbggsignalmodel = False
        if len(zzggsignalmodel)>0:
            if zzggsignaltxt:
                zzggsignalmodel = True
            else:
                zzggsignalmodel = False
        else:
            zzggsignalmodel = False
        if not os.path.exists(mass_dir):
            os.makedirs(mass_dir)
        destination_text_path = "/afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/2018datacard/" + mass_dir + "/Datacard_" + mass + "_" + cat + ".txt"
        if os.path.exists(destination_text_path):
            os.remove(destination_text_path) 
        print(destination_text_path)
        # print(FHSLsignalmodel)
        # print(bbggsignalmodel)
        # print(zzggsignalmodel)
        if FHSLsignalmodel and not bbggsignalmodel and not zzggsignalmodel:
            print("FHSL",mass,cat)
            with open(FHSLcat_path, 'r') as source_file:
                        content = source_file.read()
            with open(destination_text_path, 'w') as destination_file:
                destination_file.write(content.replace("gghh_2018_hgg", "gghhWWgg_2018_hgg"))
        if FHSLsignalmodel and bbggsignalmodel and not zzggsignalmodel:
            print("FHSL and bbgg",mass,cat)
            with open(FHSLcat_path, 'r') as source_file:
                        content = source_file.read()
            with open(destination_text_path, 'w') as destination_file:
                destination_file.write(content.replace("gghh_2018_hgg", "gghhWWgg_2018_hgg"))
            separator = r'(?<!\*)\s+'

            df_FHSL = pd.read_table(FHSLcat_path, sep=separator, header=None, engine='python')
            df_FHSL = df_FHSL.replace('gghh_2018_hgg', 'gghhWWgg_2018_hgg')
            df_bbgg = pd.read_table(bbggcat_path, sep=separator, header=None, engine='python')
            df_bbgg = df_bbgg.replace('gghh_2018_hgg', 'gghhbbgg_2018_hgg')
            # df_zzgg = pd.read_table(zzggcat_path, sep=separator, header=None, engine='python')
            # df_zzgg = df_zzgg.replace('gghh_2018_hgg', 'gghhzzgg_2018_hgg')
            with open(destination_text_path, 'r') as file:
                lines = file.readlines()
                lines.insert(10, add_all_line(df_bbgg, 8).replace(" bbgg_"+cat+"_2018 "," combineFHSL_"+cat+"_2018 ") + "\n")
                lines[18]=(add_all_line(df_FHSL,15) + df_bbgg.iloc[15][1].replace("bbgg_"+cat+"_2018","combineFHSL_"+cat+"_2018")+ "\n")
                lines[19]=(add_all_line(df_FHSL,16) + df_bbgg.iloc[16][1]+ "\n")
                lines[20]=(add_all_line(df_FHSL,17) + str(int(df_bbgg.iloc[17][1])-1)+ "\n")
                lines[21]=(add_all_line(df_FHSL,18) + df_bbgg.iloc[18][1]+ "\n")
                lines[24]=(add_all_line(df_FHSL,20) + df_bbgg.iloc[20][2]+ "\n")
                lines[25]=(add_all_line(df_FHSL,21) + df_bbgg.iloc[21][2]+ "\n")
                lines[26]=(add_all_line(df_FHSL,22) + df_bbgg.iloc[22][2]+ "\n")
                lines[27]=(add_all_line(df_FHSL,23) + df_bbgg.iloc[23][2]+ "\n")
                lines[28]=(add_all_line(df_FHSL,24) + df_bbgg.iloc[24][2]+ "\n")
                lines[29]=(add_all_line(df_FHSL,25) + df_bbgg.iloc[25][2]+ "\n")
                lines[30]=(add_all_line(df_FHSL,26) + df_bbgg.iloc[26][2]+ "\n")
                lines[31]=(add_all_line(df_FHSL,27) + df_bbgg.iloc[27][2]+ "\n")
                lines[32]=(add_all_line(df_FHSL,28) + df_bbgg.iloc[28][2]+ "\n")
                lines[33]=(add_all_line(df_FHSL,29) + df_bbgg.iloc[29][2]+ "\n")
                lines[34]=(add_all_line(df_FHSL,30) + df_bbgg.iloc[30][2]+ "\n")
                lines[35]=(add_all_line(df_FHSL,31) + df_bbgg.iloc[31][2]+ "\n")
                lines[36]=(add_all_line(df_FHSL,32) + df_bbgg.iloc[32][2]+ "\n")
                lines[37]=(add_all_line(df_FHSL,33) + df_bbgg.iloc[33][2]+ "\n")
                lines[38]=(add_all_line(df_FHSL,34) + df_bbgg.iloc[34][2]+ "\n")
                lines[39]=(add_all_line(df_FHSL,35) + df_bbgg.iloc[35][2]+ "\n")
                lines[40]=(add_all_line(df_FHSL,36) + df_bbgg.iloc[36][2]+ "\n")
                lines[41]=(add_all_line(df_FHSL,37) + df_bbgg.iloc[37][2]+ "\n")
                lines[42]=(add_all_line(df_FHSL,38) + df_bbgg.iloc[38][2]+ "\n")
                lines[43]=(add_all_line(df_FHSL,39) + df_bbgg.iloc[39][2]+ "\n")
                lines[44]=(add_all_line(df_FHSL,40) + df_bbgg.iloc[40][2]+ "\n")
                lines[45]=(add_all_line(df_FHSL,41) + df_bbgg.iloc[41][2]+ "\n")
                # lines[46]=(add_all_line(df_FHSL,42) + df_bbgg.iloc[42][2]+ "\n")
                lines[47]=(add_all_line(df_FHSL,43) + df_bbgg.iloc[43][2]+ "\n")
                lines[48]=(add_all_line(df_FHSL,44) + df_bbgg.iloc[44][2]+ "\n")
                # lines[49]=(add_all_line(df_FHSL,45) + df_bbgg.iloc[45][2]+ "\n")
                
                with open(destination_text_path, 'w') as file:
                    file.writelines(lines)
        if FHSLsignalmodel and bbggsignalmodel and zzggsignalmodel:
            print("FHSL, bbgg, zzgg ",mass,cat)
            with open(FHSLcat_path, 'r') as source_file:
                        content = source_file.read()
            with open(destination_text_path, 'w') as destination_file:
                destination_file.write(content.replace("gghh_2018_hgg", "gghhWWgg_2018_hgg"))
            separator = r'(?<!\*)\s+'

            df_FHSL = pd.read_table(FHSLcat_path, sep=separator, header=None, engine='python')
            df_FHSL = df_FHSL.replace('gghh_2018_hgg', 'gghhWWgg_2018_hgg')
            df_bbgg = pd.read_table(bbggcat_path, sep=separator, header=None, engine='python')
            df_bbgg = df_bbgg.replace('gghh_2018_hgg', 'gghhbbgg_2018_hgg')
            df_zzgg = pd.read_table(zzggcat_path, sep=separator, header=None, engine='python')
            df_zzgg = df_zzgg.replace('gghh_2018_hgg', 'gghhzzgg_2018_hgg')
            with open(destination_text_path, 'r') as file:
                lines = file.readlines()
                lines.insert(10, add_all_line(df_bbgg, 8).replace(" bbgg_"+cat+"_2018 "," combineFHSL_"+cat+"_2018 ") + "\n")
                lines[18]=(add_all_line(df_FHSL,15) + df_bbgg.iloc[15][1].replace("bbgg_"+cat+"_2018","combineFHSL_"+cat+"_2018")+"     "+df_zzgg.iloc[15][1].replace("zzgg_"+cat+"_2018","combineFHSL_"+cat+"_2018")+ "\n")
                lines[19]=(add_all_line(df_FHSL,16) + df_bbgg.iloc[16][1]+"     "+df_zzgg.iloc[16][1]+ "\n")
                lines[20]=(add_all_line(df_FHSL,17) + str(int(df_bbgg.iloc[17][1])-1)+"     " +str(int(df_bbgg.iloc[17][1])-2)+ "\n")
                lines[21]=(add_all_line(df_FHSL,18) + df_bbgg.iloc[18][1]+"     "+df_zzgg.iloc[18][1]+ "\n")
                lines[24]=(add_all_line(df_FHSL,20) + df_bbgg.iloc[20][2]+"     "+df_zzgg.iloc[20][2]+ "\n")
                lines[25]=(add_all_line(df_FHSL,21) + df_bbgg.iloc[21][2]+"     "+df_zzgg.iloc[21][2]+ "\n")
                lines[26]=(add_all_line(df_FHSL,22) + df_bbgg.iloc[22][2]+"     "+df_zzgg.iloc[22][2]+ "\n")
                lines[27]=(add_all_line(df_FHSL,23) + df_bbgg.iloc[23][2]+"     "+df_zzgg.iloc[23][2]+ "\n")
                lines[28]=(add_all_line(df_FHSL,24) + df_bbgg.iloc[24][2]+"     "+df_zzgg.iloc[24][2]+ "\n")
                lines[29]=(add_all_line(df_FHSL,25) + df_bbgg.iloc[25][2]+"     "+df_zzgg.iloc[25][2]+ "\n")
                lines[30]=(add_all_line(df_FHSL,26) + df_bbgg.iloc[26][2]+"     "+df_zzgg.iloc[26][2]+ "\n")
                lines[31]=(add_all_line(df_FHSL,27) + df_bbgg.iloc[27][2]+"     "+df_zzgg.iloc[27][2]+ "\n")
                lines[32]=(add_all_line(df_FHSL,28) + df_bbgg.iloc[28][2]+"     "+df_zzgg.iloc[28][2]+ "\n")
                lines[33]=(add_all_line(df_FHSL,29) + df_bbgg.iloc[29][2]+"     "+df_zzgg.iloc[29][2]+ "\n")
                lines[34]=(add_all_line(df_FHSL,30) + df_bbgg.iloc[30][2]+"     "+df_zzgg.iloc[30][2]+ "\n")
                lines[35]=(add_all_line(df_FHSL,31) + df_bbgg.iloc[31][2]+"     "+df_zzgg.iloc[31][2]+ "\n")
                lines[36]=(add_all_line(df_FHSL,32) + df_bbgg.iloc[32][2]+"     "+df_zzgg.iloc[32][2]+ "\n")
                lines[37]=(add_all_line(df_FHSL,33) + df_bbgg.iloc[33][2]+"     "+df_zzgg.iloc[33][2]+ "\n")
                lines[38]=(add_all_line(df_FHSL,34) + df_bbgg.iloc[34][2]+"     "+df_zzgg.iloc[34][2]+ "\n")
                lines[39]=(add_all_line(df_FHSL,35) + df_bbgg.iloc[35][2]+"     "+df_zzgg.iloc[35][2]+ "\n")
                lines[40]=(add_all_line(df_FHSL,36) + df_bbgg.iloc[36][2]+"     "+df_zzgg.iloc[36][2]+ "\n")
                lines[41]=(add_all_line(df_FHSL,37) + df_bbgg.iloc[37][2]+"     "+df_zzgg.iloc[37][2]+ "\n")
                lines[42]=(add_all_line(df_FHSL,38) + df_bbgg.iloc[38][2]+"     "+df_zzgg.iloc[38][2]+ "\n")
                lines[43]=(add_all_line(df_FHSL,39) + df_bbgg.iloc[39][2]+"     "+df_zzgg.iloc[39][2]+ "\n")
                lines[44]=(add_all_line(df_FHSL,40) + df_bbgg.iloc[40][2]+"     "+df_zzgg.iloc[40][2]+ "\n")
                lines[45]=(add_all_line(df_FHSL,41) + df_bbgg.iloc[41][2]+"     "+df_zzgg.iloc[41][2]+ "\n")
                # lines[46]=(add_all_line(df_FHSL,42) + df_bbgg.iloc[42][2]+"     "+df_zzgg.iloc[42][2]+ "\n")
                lines[47]=(add_all_line(df_FHSL,43) + df_bbgg.iloc[43][2]+"     "+df_zzgg.iloc[43][2]+ "\n")
                lines[48]=(add_all_line(df_FHSL,44) + df_bbgg.iloc[44][2]+"     "+df_zzgg.iloc[44][2]+ "\n")
                # lines[49]=(add_all_line(df_FHSL,45) + df_bbgg.iloc[45][2]+"     "+df_zzgg.iloc[45][2]+ "\n")
                lines.insert(10, add_all_line(df_zzgg, 8).replace(" zzgg_"+cat+"_2018 "," combineFHSL_"+cat+"_2018 ") + "\n")
                with open(destination_text_path, 'w') as file:
                    file.writelines(lines)
        if FHSLsignalmodel and zzggsignalmodel and not bbggsignalmodel:
            print("FHSL and zzgg",mass,cat)
            with open(FHSLcat_path, 'r') as source_file:
                        content = source_file.read()
            with open(destination_text_path, 'w') as destination_file:
                destination_file.write(content.replace("gghh_2018_hgg", "gghhWWgg_2018_hgg"))
            separator = r'(?<!\*)\s+'

            df_FHSL = pd.read_table(FHSLcat_path, sep=separator, header=None, engine='python')
            df_FHSL = df_FHSL.replace('gghh_2018_hgg', 'gghhWWgg_2018_hgg')
            # df_bbgg = pd.read_table(bbggcat_path, sep=separator, header=None, engine='python')
            # df_bbgg = df_bbgg.replace('gghh_2018_hgg', 'gghhbbgg_2018_hgg')
            df_zzgg = pd.read_table(zzggcat_path, sep=separator, header=None, engine='python')
            df_zzgg = df_zzgg.replace('gghh_2018_hgg', 'gghhzzgg_2018_hgg')
            with open(destination_text_path, 'r') as file:
                lines = file.readlines()
                lines.insert(10, add_all_line(df_zzgg, 8).replace(" zzgg_"+cat+"_2018 "," combineFHSL_"+cat+"_2018 ") + "\n")
                lines[18]=(add_all_line(df_FHSL,15) + df_zzgg.iloc[15][1].replace("zzgg_"+cat+"_2018","combineFHSL_"+cat+"_2018")+ "\n")
                lines[19]=(add_all_line(df_FHSL,16) + df_zzgg.iloc[16][1]+ "\n")
                lines[20]=(add_all_line(df_FHSL,17) + str(int(df_zzgg.iloc[17][1])-1)+ "\n")
                lines[21]=(add_all_line(df_FHSL,18) + df_zzgg.iloc[18][1]+ "\n")
                lines[24]=(add_all_line(df_FHSL,20) + df_zzgg.iloc[20][2]+ "\n")
                lines[25]=(add_all_line(df_FHSL,21) + df_zzgg.iloc[21][2]+ "\n")
                lines[26]=(add_all_line(df_FHSL,22) + df_zzgg.iloc[22][2]+ "\n")
                lines[27]=(add_all_line(df_FHSL,23) + df_zzgg.iloc[23][2]+ "\n")
                lines[28]=(add_all_line(df_FHSL,24) + df_zzgg.iloc[24][2]+ "\n")
                lines[29]=(add_all_line(df_FHSL,25) + df_zzgg.iloc[25][2]+ "\n")
                lines[30]=(add_all_line(df_FHSL,26) + df_zzgg.iloc[26][2]+ "\n")
                lines[31]=(add_all_line(df_FHSL,27) + df_zzgg.iloc[27][2]+ "\n")
                lines[32]=(add_all_line(df_FHSL,28) + df_zzgg.iloc[28][2]+ "\n")
                lines[33]=(add_all_line(df_FHSL,29) + df_zzgg.iloc[29][2]+ "\n")
                lines[34]=(add_all_line(df_FHSL,30) + df_zzgg.iloc[30][2]+ "\n")
                lines[35]=(add_all_line(df_FHSL,31) + df_zzgg.iloc[31][2]+ "\n")
                lines[36]=(add_all_line(df_FHSL,32) + df_zzgg.iloc[32][2]+ "\n")
                lines[37]=(add_all_line(df_FHSL,33) + df_zzgg.iloc[33][2]+ "\n")
                lines[38]=(add_all_line(df_FHSL,34) + df_zzgg.iloc[34][2]+ "\n")
                lines[39]=(add_all_line(df_FHSL,35) + df_zzgg.iloc[35][2]+ "\n")
                lines[40]=(add_all_line(df_FHSL,36) + df_zzgg.iloc[36][2]+ "\n")
                lines[41]=(add_all_line(df_FHSL,37) + df_zzgg.iloc[37][2]+ "\n")
                lines[42]=(add_all_line(df_FHSL,38) + df_zzgg.iloc[38][2]+ "\n")
                lines[43]=(add_all_line(df_FHSL,39) + df_zzgg.iloc[39][2]+ "\n")
                lines[44]=(add_all_line(df_FHSL,40) + df_zzgg.iloc[40][2]+ "\n")
                lines[45]=(add_all_line(df_FHSL,41) + df_zzgg.iloc[41][2]+ "\n")
                # lines[46]=(add_all_line(df_FHSL,42) + df_zzgg.iloc[42][2]+ "\n")
                lines[47]=(add_all_line(df_FHSL,43) + df_zzgg.iloc[43][2]+ "\n")
                lines[48]=(add_all_line(df_FHSL,44) + df_zzgg.iloc[44][2]+ "\n")
                # lines[49]=(add_all_line(df_FHSL,45) + df_zzgg.iloc[45][2]+ "\n")
                with open(destination_text_path, 'w') as file:
                    file.writelines(lines)

MX250_MH125 cat34FHlowpurity
/afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/2018datacard/datacard_MX250_MH125/Datacard_MX250_MH125_cat34FHlowpurity.txt
FHSL, bbgg, zzgg  MX250_MH125 cat34FHlowpurity
MX260_MH125 cat34FHlowpurity
/afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/2018datacard/datacard_MX260_MH125/Datacard_MX260_MH125_cat34FHlowpurity.txt
FHSL, bbgg, zzgg  MX260_MH125 cat34FHlowpurity
MX270_MH125 cat34FHlowpurity
/afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/2018datacard/datacard_MX270_MH125/Datacard_MX270_MH125_cat34FHlowpurity.txt
FHSL, bbgg, zzgg  MX270_MH125 cat34FHlowpurity
MX280_MH125 cat34FHlowpurity
/afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/2018datacard/datacard_MX280_MH125/Datacard_MX280_MH125_cat34FHlowpurity.txt
FHSL, bbgg, zzgg  MX280_MH125 cat34FHlowpurity
MX300_MH125 cat34FHlowpurity
/afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/2018datacard/

In [8]:
cat_name = ['cat12highpurity','cat12lowpurity']
mass_name=['MX1000_MH125','MX500_MH125','MX550_MH125','MX600_MH125','MX650_MH125','MX700_MH125','MX750_MH125','MX800_MH125','MX850_MH125', 'MX1100_MH125', 'MX1200_MH125', 'MX1300_MH125','MX1400_MH125','MX1500_MH125', 'MX1600_MH125', 'MX1700_MH125','MX1800_MH125','MX1900_MH125', 'MX2000_MH125', 'MX2200_MH125', 'MX2600_MH125', 'MX2800_MH125','MX2400_MH125','MX3000_MH125']
#combine command
getlimit=[]
Combinecommand=[]
txttoroot=[]
cd = []
for cat in cat_name:
    for mass in mass_name: 
        mass_dir = "datacard_" + mass
        FHSLcat_path="/afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/2018datacard/"+mass_dir+"/Datacard_"+mass+"_2018_combineFHSL_"+cat+".txt"
        bbggcat_path="/afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/2018datacard/"+mass_dir+"/Datacard_"+mass+"_2018_bbgg_"+cat+".txt"
        zzggcat_path="/afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/2018datacard/"+mass_dir+"/Datacard_"+mass+"_2018_zzgg_"+cat+".txt"
        FHSLws_path="/eos/cms/store/group/phys_b2g/shsong/flashggws/cat12/2018/"+mass+"/ws_gghh_"+mass+"_2018_combineFHSL_"+cat
        bbggws_path="/eos/cms/store/group/phys_b2g/shsong/flashggws/cat12/2018/"+mass+"/ws_gghh_"+mass+"_2018_bbgg_"+cat
        zzggws_path="/eos/cms/store/group/phys_b2g/shsong/flashggws/cat12/2018/"+mass+"/ws_gghh_"+mass+"_2018_zzgg_"+cat
        FHSLsignalmodel= glob.glob(os.path.join(FHSLws_path, 'CMS-HGG_sigfit_packaged_*.root'))
        bbggsignalmodel= glob.glob(os.path.join(bbggws_path, 'CMS-HGG_sigfit_packaged_*.root'))
        zzggsignalmodel= glob.glob(os.path.join(zzggws_path, 'CMS-HGG_sigfit_packaged_*.root'))
        FHSLsignaltxt = os.path.getsize(FHSLcat_path) > 1000
        bbggsignaltxt = os.path.getsize(bbggcat_path) > 1000
        zzggsignaltxt = os.path.getsize(zzggcat_path) > 1000
        if len(FHSLsignalmodel)>0:
            if FHSLsignaltxt:
                FHSLsignalmodel = True
            else:
                FHSLsignalmodel = False
        else:
            FHSLsignalmodel = False
        if len(bbggsignalmodel)>0:
            if bbggsignaltxt:
                bbggsignalmodel = True
            else:
                bbggsignalmodel = False
        else:
            bbggsignalmodel = False
        if len(zzggsignalmodel)>0:
            if zzggsignaltxt:
                zzggsignalmodel = True
            else:
                zzggsignalmodel = False
        else:
            zzggsignalmodel = False

        if not os.path.exists(mass_dir):
            os.makedirs(mass_dir)
        destination_text_path = "/afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/2018datacard/" + mass_dir + "/Datacard_" + mass + "_" + cat + ".txt"
        if os.path.exists(destination_text_path):
            os.remove(destination_text_path) 
        print(destination_text_path)
        print(FHSLsignalmodel)
        print(bbggsignalmodel)
        print(zzggsignalmodel)
        if FHSLsignalmodel and bbggsignalmodel and not zzggsignalmodel:
            print("FHSL and bbgg",mass,cat)
            with open(FHSLcat_path, 'r') as source_file:
                        content = source_file.read()
            with open(destination_text_path, 'w') as destination_file:
                destination_file.write(content.replace("gghh_2018_hgg", "gghhWWgg_2018_hgg"))
            separator = r'(?<!\*)\s+'

            df_FHSL = pd.read_table(FHSLcat_path, sep=separator, header=None, engine='python')
            df_FHSL = df_FHSL.replace('gghh_2018_hgg', 'gghhWWgg_2018_hgg')
            df_bbgg = pd.read_table(bbggcat_path, sep=separator, header=None, engine='python')
            df_bbgg = df_bbgg.replace('gghh_2018_hgg', 'gghhbbgg_2018_hgg')
            df_zzgg = pd.read_table(zzggcat_path, sep=separator, header=None, engine='python')
            df_zzgg = df_zzgg.replace('gghh_2018_hgg', 'gghhzzgg_2018_hgg')
            with open(destination_text_path, 'r') as file:
                lines = file.readlines()
                lines.insert(10, add_all_line(df_bbgg, 8).replace(" bbgg_"+cat+"_2018 "," combineFHSL_"+cat+"_2018 ") + "\n")
                lines[18]=(add_all_line(df_FHSL,15) + df_bbgg.iloc[15][1].replace("bbgg_"+cat+"_2018","combineFHSL_"+cat+"_2018")+ "\n")
                lines[19]=(add_all_line(df_FHSL,16) + df_bbgg.iloc[16][1]+ "\n")
                lines[20]=(add_all_line(df_FHSL,17) + str(int(df_bbgg.iloc[17][1])-1)+ "\n")
                lines[21]=(add_all_line(df_FHSL,18) + df_bbgg.iloc[18][1]+ "\n")
                lines[24]=(add_all_line(df_FHSL,20) + df_bbgg.iloc[20][2]+ "\n")
                lines[25]=(add_all_line(df_FHSL,21) + df_bbgg.iloc[21][2]+ "\n")
                lines[26]=(add_all_line(df_FHSL,22) + df_bbgg.iloc[22][2]+ "\n")
                lines[27]=(add_all_line(df_FHSL,23) + df_bbgg.iloc[23][2]+ "\n")
                lines[28]=(add_all_line(df_FHSL,24) + df_bbgg.iloc[24][2]+ "\n")
                lines[29]=(add_all_line(df_FHSL,25) + df_bbgg.iloc[25][2]+ "\n")
                lines[30]=(add_all_line(df_FHSL,26) + df_bbgg.iloc[26][2]+ "\n")
                lines[31]=(add_all_line(df_FHSL,27) + df_bbgg.iloc[27][2]+ "\n")
                lines[32]=(add_all_line(df_FHSL,28) + df_bbgg.iloc[28][2]+ "\n")
                lines[33]=(add_all_line(df_FHSL,29) + df_bbgg.iloc[29][2]+ "\n")
                lines[34]=(add_all_line(df_FHSL,30) + df_bbgg.iloc[30][2]+ "\n")
                lines[35]=(add_all_line(df_FHSL,31) + df_bbgg.iloc[31][2]+ "\n")
                lines[36]=(add_all_line(df_FHSL,32) + df_bbgg.iloc[32][2]+ "\n")
                lines[37]=(add_all_line(df_FHSL,33) + df_bbgg.iloc[33][2]+ "\n")
                lines[38]=(add_all_line(df_FHSL,34) + df_bbgg.iloc[34][2]+ "\n")
                lines[39]=(add_all_line(df_FHSL,35) + df_bbgg.iloc[35][2]+ "\n")
                lines[40]=(add_all_line(df_FHSL,36) + df_bbgg.iloc[36][2]+ "\n")
                lines[41]=(add_all_line(df_FHSL,37) + df_bbgg.iloc[37][2]+ "\n")
                lines[42]=(add_all_line(df_FHSL,38) + df_bbgg.iloc[38][2]+ "\n")
                lines[43]=(add_all_line(df_FHSL,39) + df_bbgg.iloc[39][2]+ "\n")
                lines[44]=(add_all_line(df_FHSL,40) + df_bbgg.iloc[40][2]+ "\n")
                lines[45]=(add_all_line(df_FHSL,41) + df_bbgg.iloc[41][2]+ "\n")
                lines[46]=(add_all_line(df_FHSL,42) + df_bbgg.iloc[42][2]+ "\n")
                lines[47]=(add_all_line(df_FHSL,43) + df_bbgg.iloc[43][2]+ "\n")
                lines[48]=(add_all_line(df_FHSL,44) + df_bbgg.iloc[44][2]+ "\n")
                # lines[49]=(add_all_line(df_FHSL,45) + df_bbgg.iloc[45][2]+ "\n")
                lines[50]=(add_all_line(df_FHSL,46) + df_bbgg.iloc[46][2]+ "\n")
                lines[51]=(add_all_line(df_FHSL,47) + df_bbgg.iloc[47][2]+ "\n")
                
                with open(destination_text_path, 'w') as file:
                    file.writelines(lines)
        if FHSLsignalmodel and bbggsignalmodel and zzggsignalmodel:
            with open(FHSLcat_path, 'r') as source_file:
                        content = source_file.read()
            with open(destination_text_path, 'w') as destination_file:
                destination_file.write(content.replace("gghh_2018_hgg", "gghhWWgg_2018_hgg"))
            separator = r'(?<!\*)\s+'
            print(FHSLcat_path)
            df_FHSL = pd.read_table(FHSLcat_path, sep=separator, header=None, engine='python')
            df_FHSL = df_FHSL.replace('gghh_2018_hgg', 'gghhWWgg_2018_hgg')
            df_bbgg = pd.read_table(bbggcat_path, sep=separator, header=None, engine='python')
            df_bbgg = df_bbgg.replace('gghh_2018_hgg', 'gghhbbgg_2018_hgg')
            df_zzgg = pd.read_table(zzggcat_path, sep=separator, header=None, engine='python')
            df_zzgg = df_zzgg.replace('gghh_2018_hgg', 'gghhzzgg_2018_hgg')
            with open(destination_text_path, 'r') as file:
                lines = file.readlines()
                lines.insert(10, add_all_line(df_bbgg, 8).replace(" bbgg_"+cat+"_2018 "," combineFHSL_"+cat+"_2018 ") + "\n")
                lines[18]=(add_all_line(df_FHSL,15) + df_bbgg.iloc[15][1].replace("bbgg_"+cat+"_2018","combineFHSL_"+cat+"_2018")+"     "+df_zzgg.iloc[15][1].replace("zzgg_"+cat+"_2018","combineFHSL_"+cat+"_2018")+ "\n")
                lines[19]=(add_all_line(df_FHSL,16) + df_bbgg.iloc[16][1]+"     "+df_zzgg.iloc[16][1]+ "\n")
                lines[20]=(add_all_line(df_FHSL,17) + str(int(df_bbgg.iloc[17][1])-1)+"     " +str(int(df_bbgg.iloc[17][1])-2)+ "\n")
                lines[21]=(add_all_line(df_FHSL,18) + df_bbgg.iloc[18][1]+"     "+df_zzgg.iloc[18][1]+ "\n")
                lines[24]=(add_all_line(df_FHSL,20) + df_bbgg.iloc[20][2]+"     "+df_zzgg.iloc[20][2]+ "\n")
                lines[25]=(add_all_line(df_FHSL,21) + df_bbgg.iloc[21][2]+"     "+df_zzgg.iloc[21][2]+ "\n")
                lines[26]=(add_all_line(df_FHSL,22) + df_bbgg.iloc[22][2]+"     "+df_zzgg.iloc[22][2]+ "\n")
                lines[27]=(add_all_line(df_FHSL,23) + df_bbgg.iloc[23][2]+"     "+df_zzgg.iloc[23][2]+ "\n")
                lines[28]=(add_all_line(df_FHSL,24) + df_bbgg.iloc[24][2]+"     "+df_zzgg.iloc[24][2]+ "\n")
                lines[29]=(add_all_line(df_FHSL,25) + df_bbgg.iloc[25][2]+"     "+df_zzgg.iloc[25][2]+ "\n")
                lines[30]=(add_all_line(df_FHSL,26) + df_bbgg.iloc[26][2]+"     "+df_zzgg.iloc[26][2]+ "\n")
                lines[31]=(add_all_line(df_FHSL,27) + df_bbgg.iloc[27][2]+"     "+df_zzgg.iloc[27][2]+ "\n")
                lines[32]=(add_all_line(df_FHSL,28) + df_bbgg.iloc[28][2]+"     "+df_zzgg.iloc[28][2]+ "\n")
                lines[33]=(add_all_line(df_FHSL,29) + df_bbgg.iloc[29][2]+"     "+df_zzgg.iloc[29][2]+ "\n")
                lines[34]=(add_all_line(df_FHSL,30) + df_bbgg.iloc[30][2]+"     "+df_zzgg.iloc[30][2]+ "\n")
                lines[35]=(add_all_line(df_FHSL,31) + df_bbgg.iloc[31][2]+"     "+df_zzgg.iloc[31][2]+ "\n")
                lines[36]=(add_all_line(df_FHSL,32) + df_bbgg.iloc[32][2]+"     "+df_zzgg.iloc[32][2]+ "\n")
                lines[37]=(add_all_line(df_FHSL,33) + df_bbgg.iloc[33][2]+"     "+df_zzgg.iloc[33][2]+ "\n")
                lines[38]=(add_all_line(df_FHSL,34) + df_bbgg.iloc[34][2]+"     "+df_zzgg.iloc[34][2]+ "\n")
                lines[39]=(add_all_line(df_FHSL,35) + df_bbgg.iloc[35][2]+"     "+df_zzgg.iloc[35][2]+ "\n")
                lines[40]=(add_all_line(df_FHSL,36) + df_bbgg.iloc[36][2]+"     "+df_zzgg.iloc[36][2]+ "\n")
                lines[41]=(add_all_line(df_FHSL,37) + df_bbgg.iloc[37][2]+"     "+df_zzgg.iloc[37][2]+ "\n")
                lines[42]=(add_all_line(df_FHSL,38) + df_bbgg.iloc[38][2]+"     "+df_zzgg.iloc[38][2]+ "\n")
                lines[43]=(add_all_line(df_FHSL,39) + df_bbgg.iloc[39][2]+"     "+df_zzgg.iloc[39][2]+ "\n")
                lines[44]=(add_all_line(df_FHSL,40) + df_bbgg.iloc[40][2]+"     "+df_zzgg.iloc[40][2]+ "\n")
                lines[45]=(add_all_line(df_FHSL,41) + df_bbgg.iloc[41][2]+"     "+df_zzgg.iloc[41][2]+ "\n")
                lines[46]=(add_all_line(df_FHSL,42) + df_bbgg.iloc[42][2]+"     "+df_zzgg.iloc[42][2]+ "\n")
                lines[47]=(add_all_line(df_FHSL,43) + df_bbgg.iloc[43][2]+"     "+df_zzgg.iloc[43][2]+ "\n")
                lines[48]=(add_all_line(df_FHSL,44) + df_bbgg.iloc[44][2]+"     "+df_zzgg.iloc[44][2]+ "\n")
                # lines[49]=(add_all_line(df_FHSL,45) + df_bbgg.iloc[45][2]+"     "+df_zzgg.iloc[45][2]+ "\n")
                lines[50]=(add_all_line(df_FHSL,46) + df_bbgg.iloc[46][2]+"     "+df_zzgg.iloc[46][2]+ "\n")
                lines[51]=(add_all_line(df_FHSL,47) + df_bbgg.iloc[47][2]+"     "+df_zzgg.iloc[47][2]+ "\n")
                lines.insert(10, add_all_line(df_zzgg, 8).replace(" zzgg_"+cat+"_2018 "," combineFHSL_"+cat+"_2018 ") + "\n")
                with open(destination_text_path, 'w') as file:
                    file.writelines(lines)
        if FHSLsignalmodel and zzggsignalmodel and not bbggsignalmodel:
            print("FHSL and zzgg",mass,cat)
            with open(FHSLcat_path, 'r') as source_file:
                        content = source_file.read()
            with open(destination_text_path, 'w') as destination_file:
                destination_file.write(content.replace("gghh_2018_hgg", "gghhWWgg_2018_hgg"))
            separator = r'(?<!\*)\s+'

            df_FHSL = pd.read_table(FHSLcat_path, sep=separator, header=None, engine='python')
            df_FHSL = df_FHSL.replace('gghh_2018_hgg', 'gghhWWgg_2018_hgg')
            df_bbgg = pd.read_table(bbggcat_path, sep=separator, header=None, engine='python')
            df_bbgg = df_bbgg.replace('gghh_2018_hgg', 'gghhbbgg_2018_hgg')
            df_zzgg = pd.read_table(zzggcat_path, sep=separator, header=None, engine='python')
            df_zzgg = df_zzgg.replace('gghh_2018_hgg', 'gghhzzgg_2018_hgg')
            with open(destination_text_path, 'r') as file:
                lines = file.readlines()
                lines.insert(10, add_all_line(df_zzgg, 8).replace(" zzgg_"+cat+"_2018 "," combineFHSL_"+cat+"_2018 ") + "\n")
                lines[18]=(add_all_line(df_FHSL,15) + df_zzgg.iloc[15][1].replace("zzgg_"+cat+"_2018","combineFHSL_"+cat+"_2018")+ "\n")
                lines[19]=(add_all_line(df_FHSL,16) + df_zzgg.iloc[16][1]+ "\n")
                lines[20]=(add_all_line(df_FHSL,17) + str(int(df_zzgg.iloc[17][1])-1)+ "\n")
                lines[21]=(add_all_line(df_FHSL,18) + df_zzgg.iloc[18][1]+ "\n")
                lines[24]=(add_all_line(df_FHSL,20) + df_zzgg.iloc[20][2]+ "\n")
                lines[25]=(add_all_line(df_FHSL,21) + df_zzgg.iloc[21][2]+ "\n")
                lines[26]=(add_all_line(df_FHSL,22) + df_zzgg.iloc[22][2]+ "\n")
                lines[27]=(add_all_line(df_FHSL,23) + df_zzgg.iloc[23][2]+ "\n")
                lines[28]=(add_all_line(df_FHSL,24) + df_zzgg.iloc[24][2]+ "\n")
                lines[29]=(add_all_line(df_FHSL,25) + df_zzgg.iloc[25][2]+ "\n")
                lines[30]=(add_all_line(df_FHSL,26) + df_zzgg.iloc[26][2]+ "\n")
                lines[31]=(add_all_line(df_FHSL,27) + df_zzgg.iloc[27][2]+ "\n")
                lines[32]=(add_all_line(df_FHSL,28) + df_zzgg.iloc[28][2]+ "\n")
                lines[33]=(add_all_line(df_FHSL,29) + df_zzgg.iloc[29][2]+ "\n")
                lines[34]=(add_all_line(df_FHSL,30) + df_zzgg.iloc[30][2]+ "\n")
                lines[35]=(add_all_line(df_FHSL,31) + df_zzgg.iloc[31][2]+ "\n")
                lines[36]=(add_all_line(df_FHSL,32) + df_zzgg.iloc[32][2]+ "\n")
                lines[37]=(add_all_line(df_FHSL,33) + df_zzgg.iloc[33][2]+ "\n")
                lines[38]=(add_all_line(df_FHSL,34) + df_zzgg.iloc[34][2]+ "\n")
                lines[39]=(add_all_line(df_FHSL,35) + df_zzgg.iloc[35][2]+ "\n")
                lines[40]=(add_all_line(df_FHSL,36) + df_zzgg.iloc[36][2]+ "\n")
                lines[41]=(add_all_line(df_FHSL,37) + df_zzgg.iloc[37][2]+ "\n")
                lines[42]=(add_all_line(df_FHSL,38) + df_zzgg.iloc[38][2]+ "\n")
                lines[43]=(add_all_line(df_FHSL,39) + df_zzgg.iloc[39][2]+ "\n")
                lines[44]=(add_all_line(df_FHSL,40) + df_zzgg.iloc[40][2]+ "\n")
                lines[45]=(add_all_line(df_FHSL,41) + df_zzgg.iloc[41][2]+ "\n")
                lines[46]=(add_all_line(df_FHSL,42) + df_zzgg.iloc[42][2]+ "\n")
                lines[47]=(add_all_line(df_FHSL,43) + df_zzgg.iloc[43][2]+ "\n")
                lines[48]=(add_all_line(df_FHSL,44) + df_zzgg.iloc[44][2]+ "\n")
                # lines[49]=(add_all_line(df_FHSL,45) + df_zzgg.iloc[45][2]+ "\n")
                lines[50]=(add_all_line(df_FHSL,46) + df_zzgg.iloc[46][2]+ "\n")
                lines[51]=(add_all_line(df_FHSL,47) + df_zzgg.iloc[47][2]+ "\n")
                with open(destination_text_path, 'w') as file:
                    file.writelines(lines)

/afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/2018datacard/datacard_MX1000_MH125/Datacard_MX1000_MH125_cat12highpurity.txt
True
True
True
/afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/2018datacard/datacard_MX1000_MH125/Datacard_MX1000_MH125_2018_combineFHSL_cat12highpurity.txt
/afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/2018datacard/datacard_MX500_MH125/Datacard_MX500_MH125_cat12highpurity.txt
True
False
True
FHSL and zzgg MX500_MH125 cat12highpurity
/afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/2018datacard/datacard_MX550_MH125/Datacard_MX550_MH125_cat12highpurity.txt
True
True
True
/afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/2018datacard/datacard_MX550_MH125/Datacard_MX550_MH125_2018_combineFHSL_cat12highpurity.txt
/afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/2018datacard/datacard_MX600_MH125/Datacard_MX600_MH125_cat12highpurity.txt
True

In [7]:
df_FHSL

,0,1,2,3,4,5,6
0,CMS,HGG,Datacard,-,2018,-,13TeV
1,Auto-generated,by,flashggFinalFits/Datacard/makeDatacard.py,None,None,None,None
2,Run,with:,combine,None,None,None,None
3,---------------------------------------------,None,None,None,None,None,None
4,imax,*,None,None,None,None,None
5,jmax,*,None,None,None,None,None
6,kmax,*,None,None,None,None,None
7,---------------------------------------------,None,None,None,None,None,None
8,shapes,gghhWWgg_2018_hgg,combineFHSL_cat12lowpurity_2018,/eos/cms/store/group/phys_b2g/shsong/flashggws...,wsig_13TeV:hggpdfsmrel_gghh_2018_combineFHSL_c...,None,None
9,shapes,bkg_mass,combineFHSL_cat12lowpurity_2018,/eos/cms/store/group/phys_b2g/shsong/flashggws...,multipdf:CMS_hgg_combineFHSL_cat12lowpurity_20...,None,None
